In [ ]:
def database_connect_oracle(db_user, db_password, user, database):
    sql_connections = []
    connection_string = db_user + '/' + db_password + database
    print("\n***CONNECTING***: " + db_user + '/' + "************" + database + "\n--------------")
    con = cx_Oracle.connect(connection_string)
    sql_connections.append(con)
    con_engine = create_engine('oracle+cx_oracle://' + db_user + ':' + db_password + database) #'@192.168.86.108:5522/?service_name=xe')
    sql_connections.append(con_engine)
    return sql_connections

In [ ]:
def database_close_oracle(connection, sql_connections):
    connection[0].close()
    print("--------------\n***DISCONNECTING***\n")

In [ ]:
def database_insert_oracle(input_df, sql_connections)
    con = sql_connections[0]
    con_engine = sql_connections[1]
    total_process = input_df.shape[0]
    invalid_category = []
    invalid_blog = []
    invalid_post = []
    #connection_string = db_user + '/' + db_password + database
    #print("\n***CONNECTING***: " + db_user + '/' + "************" + database + "\n--------------")
    #con = cx_Oracle.connect(connection_string)
    #con_engine = create_engine('oracle+cx_oracle://' + db_user + ':' + db_password + '@192.168.86.108:5522/?service_name=xe')
    
    c = 0
    ce = 0
    b = 0
    be = 0
    p = 0
    pdup = 0
    pe = 0
    
    #(1) COMPARE TO EXISTING CATEGORIES:
    for i, row in enumerate(input_df.itertuples(), 1): 
        try:
            beliefs_test = pd.read_sql("SELECT * FROM patheos_beliefs WHERE beliefs_url = '" + str(row.beliefs_url) + "'", con_engine)
            if beliefs_test.empty == True:
                cur = con.cursor()
                cur.execute("SELECT MAX(beliefs_number) FROM patheos_beliefs")
                test_number = str(cur.fetchall()[0][0])
                if test_number != 'None':
                    new_number = int(test_number) + 1
                else:
                    new_number = 1                
                beliefs_number = str(new_number)
                cur.execute("""INSERT INTO patheos_beliefs
                                (BELIEFS_NUMBER, BELIEFS_NAME, BELIEFS_URL)
                                VALUES
                                ('""" + str(new_number) + """','""" + row.beliefs_name + """','""" + row.beliefs_url + """')""")
                con.commit()
                cur2 = con.cursor()
                cur2.execute("SELECT * FROM patheos_beliefs WHERE beliefs_number = '" + str(new_number) + "'")
                res = cur2.fetchall()
                cur2.close()
                cur.close()
                c = c + 1
            else:
                cur3 = con.cursor()
                cur3.execute("SELECT MAX(beliefs_number) FROM patheos_beliefs WHERE beliefs_url = '" + row.beliefs_url + "'")
                beliefs_number = str(cur3.fetchall()[0][0])
                cur3.close()
        except:
            invalid_category.append(row.beliefs_url)
            ce = ce + 1 
        
        
    #(2) COMPARE TO EXISTING BLOGS:
        try:
            blogs_test = pd.read_sql("SELECT * FROM patheos_blogs WHERE blogs_url = '" + str(row.blogs_url) + "'", con_engine) 
                                        # beliefs_number = " + beliefs_number + " AND posts_url = '" + str(row.blogs_url) + "'", con_engine)
            if blogs_test.empty == True:
                cur4 = con.cursor()
                cur4.execute("SELECT MAX(blogs_number) FROM patheos_blogs")
                test_number2 = str(cur4.fetchall()[0][0])
                if test_number2 != 'None':
                    new_blog_number = int(test_number2) + 1
                else:
                    new_blog_number = 1                
                blogs_number = str(new_blog_number)
                cur4.execute("""INSERT INTO patheos_blogs
                                (BLOGS_NUMBER, BELIEFS_NUMBER, BLOGS_NAME, BLOGS_URL)
                                VALUES
                                ('""" + str(new_blog_number) + """','""" + beliefs_number + """','""" + row.blogs_name + """','""" + row.blogs_url + """')""")           
                con.commit()
                cur5 = con.cursor()
                cur5.execute("SELECT * FROM patheos_blogs WHERE blogs_number = '" + str(new_blog_number) + "'")
                res2 = cur5.fetchall()
                cur5.close()
                cur4.close()
                b = b + 1
            else:
                cur6 = con.cursor()
                cur6.execute("SELECT MAX(blogs_number) FROM patheos_blogs WHERE blogs_url = '" + row.blogs_url + "'")
                blogs_number = str(cur6.fetchall()[0][0])
                cur6.close()
        except:
            invalid_blog.append(row.blogs_url)
            be = be + 1
        
    #(3) COMPARE TO EXISTING POSTS:
        try:
            posts_test = pd.read_sql("SELECT * FROM patheos_posts WHERE posts_url = '" + str(row.posts_url) + "'", con_engine) 
                                    # beliefs_number = " + beliefs_number + " AND posts_url = '" + str(row.blogs_url) + "'", con_engine)
            if posts_test.empty == True:
                cur7 = con.cursor()
                cur7.execute("SELECT MAX(posts_number) FROM patheos_posts")
                test_number3 = str(cur7.fetchall()[0][0])
                if test_number3 != 'None':
                    new_post_number = int(test_number3) + 1
                else:
                    new_post_number = 1    
                var = cur7.var(cx_Oracle.CLOB)
                var.setvalue(0, row.posts_content)    # write a small value first to force the temporary LOB to be created    
                cur7.execute("""INSERT INTO patheos_posts
                                (POSTS_NUMBER, POSTS_TITLE, BLOGS_NUMBER, POSTS_AUTHOR, POSTS_DATE, POSTS_CONTENT, POSTS_URL)
                                VALUES
                                ('""" + str(new_post_number) + """','""" + row.posts_title + """','""" + blogs_number + """','""" + row.posts_author + """',""" 
                                      + """to_date('""" + row.posts_date + """','MONTH DD, YYYY'), :val ,'""" + row.posts_url + """')""", val = var)   
                con.commit()
                #cur8 = con.cursor()
                #cur8.execute("SELECT * FROM patheos_posts WHERE posts_number = '" + str(new_post_number) + "'")
                #res3 = cur8.fetchall()
                #cur8.close()
                #print("INSERTED POST: " + str(res3) + "\n")
                cur7.close()
                p = p + 1
                return_code = 1
            else:
                cur9 = con.cursor()
                cur9.execute("SELECT MAX(posts_number) FROM patheos_posts WHERE posts_url = '" + row.posts_url + "'")
                posts_number = str(cur9.fetchall()[0][0])
                cur9.close()
                pdup = pdup + 1
                return_code = 2
        except:
            invalid_post.append(row.posts_url)
            pe = pe + 1
            return_code = 0
        sys.stdout.write('\r'+ str(i) + "/" + str(total_process) + " | BELIEFS: new(" + str(c) + ") e(" + str(ce) 
                                                                 + ") | BLOGS: new(" + str(b) + ") e(" + str(be) 
                                                                 + ") | POSTS: new(" + str(p) + ") dup(" + str(pdup) + ") e(" + str(pe) + ")")
        #time.sleep(0.5)
        

    print("\nINSERT COMPLETE.")
    return new_post_number
    #con.close()
    #print("--------------\n***DISCONNECTING***\n")

In [ ]:
def find_blog_pk(url, sql_connections):
    con = sql_connections[0]
    con_engine = sql_connections[1]
    blog_number = pd.read_sql("SELECT blogs_number FROM patheos_blogs WHERE blogs_url = '" + str(row.blogs_url) + "'", con_engine)
    if blog_number.empty == False:
        pk = blog_number
    else:
        pk = -1
    return pk